In [1]:
!pip install spacy

In [2]:
# !python -m spacy download ru_core_news_lg -q

## Data Loading and Preparation

In [3]:
import spacy
import json

def load_data(file_path):
    data = []
    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            data.append(json.loads(line))
    return data

data = load_data('../train.jsonl')

In [4]:
def prepare_ner_data(train_data):
    """ Prepare the training data for NER model by sorting entities, adjusting boundaries, and removing overlaps. """
    prepared_data = []
    for data in train_data:
        sentences = data['sentences']
        # Sort entities by length, prioritize longer entities first
        entities = sorted(data['ners'], key=lambda x: x[1] - x[0], reverse=True)
        
        # Adjust end boundary by adding one
        adjusted_entities = [(start, end + 1, label) for start, end, label in entities]
        
        # Remove overlapping entities
        span = set()
        non_overlapping_entities = []
        for start, end, label in adjusted_entities:
            if not any(i in span for i in range(start, end)):
                non_overlapping_entities.append((start, end, label))
                span.update(range(start, end))
        
        prepared_data.append((sentences, {'entities': non_overlapping_entities}))
    
    return prepared_data

training_data = prepare_ner_data(data)

In [5]:
print(training_data[0][0])
print(training_data[0][1])

Бостон взорвали Тамерлан и Джохар Царнаевы из Северного Кавказа

19 апреля 2013 года в пригороде Бостона  проходит спецоперация по поимке 19-летнего Джохара Царнаева, подозреваемого в теракте на Бостонском марафоне 15 апреля и в смертельном ранении полицейского на кампусе Массачусетского технологического института 18 апреля.

Второй подозреваемый, его брат, 26-летний Тамерлан Царнаев, был ранен в перестрелке в Уотертауне  и позже скончался в больнице.

Уотертаун и его окрестности фактически переведены на осадное положение: окрестности оцеплены, дороги перекрыты, магазины и бизнесы закрыты, жителей просят не выходить из домов и не приближаться к окнам, над районом спецоперации перекрыты полёты авиации.

В Бостоне приостановлена работа общественного транспорта, включая метро, автобусы, такси и пригородные поезда. Отменены занятия в Гарварде, Массачусетском технологическом институте, Университете Саффолка, Бостонском университете и во всех городских школах.

На сайте ФБР опубликованы фото

## Model Setup & Training

In [6]:
from spacy.training import Example
import random

nlp = spacy.load("ru_core_news_lg")

optimizer = nlp.initialize()

# Training loop
for itn in range(50):
    random.shuffle(training_data)
    losses = {}
    for sentences, annotations in training_data:
        doc = nlp.make_doc(sentences)
        example = Example.from_dict(doc, annotations)
        nlp.update([example], sgd=optimizer, drop=0.5, losses=losses)
    print(f"Iteration {itn}, Losses: {losses}")

c:\Users\mosta\miniconda3\Lib\site-packages\spacy\training\iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "Суд Вены освободил Дмитрия Фирташа

Дмитрий Фирташ..." with entities "[(901, 972, 'CRIME'), (1128, 1156, 'ORGANIZATION')...". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
c:\Users\mosta\miniconda3\Lib\site-packages\spacy\training\iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "В Португалии объявлен трёхдневный траур в память о..." with entities "[(1104, 1139, 'AWARD'), (466, 494, 'EVENT'), (873,...". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
c:\Users\mosta\miniconda3\Lib\site-packages\spacy\training\iob_utils.py:149: UserWarning: [W030] Some entities c

Iteration 0, Losses: {'tok2vec': 0.0, 'morphologizer': 0.0, 'parser': 0.0, 'ner': 43708.101333828265}
Iteration 1, Losses: {'tok2vec': 0.0, 'morphologizer': 0.0, 'parser': 0.0, 'ner': 31702.493964091624}
Iteration 2, Losses: {'tok2vec': 0.0, 'morphologizer': 0.0, 'parser': 0.0, 'ner': 27906.942871613835}
Iteration 3, Losses: {'tok2vec': 0.0, 'morphologizer': 0.0, 'parser': 0.0, 'ner': 25234.671167658827}
Iteration 4, Losses: {'tok2vec': 0.0, 'morphologizer': 0.0, 'parser': 0.0, 'ner': 23747.006201757726}
Iteration 5, Losses: {'tok2vec': 0.0, 'morphologizer': 0.0, 'parser': 0.0, 'ner': 22357.67307013378}
Iteration 6, Losses: {'tok2vec': 0.0, 'morphologizer': 0.0, 'parser': 0.0, 'ner': 21377.868525916852}
Iteration 7, Losses: {'tok2vec': 0.0, 'morphologizer': 0.0, 'parser': 0.0, 'ner': 20186.613170554152}
Iteration 8, Losses: {'tok2vec': 0.0, 'morphologizer': 0.0, 'parser': 0.0, 'ner': 19790.36949715096}
Iteration 9, Losses: {'tok2vec': 0.0, 'morphologizer': 0.0, 'parser': 0.0, 'ner': 19

In [9]:
# Save the model
nlp.to_disk('ner_model')

## Test Data Prediction

In [7]:
test_data = load_data('../test.jsonl')

In [8]:
def predict_and_save_entities(test_data, output_file_path):
    results = []
    for item in test_data:
        doc = nlp(item['senences'])
        ners = [(ent.start_char, ent.end_char, ent.label_) for ent in doc.ents]
        results.append({'ners': ners, 'id': item['id']})
    
    with open(output_file_path, 'w', encoding='utf-8') as file:
        for result in results:
            file.write(json.dumps(result) + '\n')

predict_and_save_entities(test_data, 'test.jsonl')

c:\Users\mosta\miniconda3\Lib\site-packages\spacy\pipeline\attributeruler.py:149: UserWarning: [W036] The component 'matcher' does not have any patterns defined.
  matches = self.matcher(doc, allow_missing=True, as_spans=False)
